<a href="https://colab.research.google.com/github/imahrukh/AI2002/blob/main/labs/lab8/lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [6]:
#load and pre process data
from google.colab import drive
drive.mount('/content/drive')

transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()  # scales pixel values to [0,1]
])

data_path = "/content/drive/MyDrive/flowers"
print("Subfolders:", os.listdir(data_path))
dataset = datasets.ImageFolder(root=data_path, transform=transform)

Mounted at /content/drive
Subfolders: ['dandelion', 'sunflower', 'tulip', 'rose', 'daisy']


FileNotFoundError: Found no valid file for the classes daisy, dandelion, rose. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
# split data
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
#Define the MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 64 * 3, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, len(dataset.classes))  # 5 output classes

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
#Training settings
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
#Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}")


In [ ]:
#model evaluation
model.eval()
y_true = []
y_pred = []
y_probs = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        _, predicted = torch.max(probs, 1)

        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())
        y_probs.extend(probs.numpy())


In [ ]:
#Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=dataset.classes))

In [ ]:
#roc curve and auc
lb = LabelBinarizer()
y_true_bin = lb.fit_transform(y_true)

fpr = {}
tpr = {}
roc_auc = {}

for i in range(len(dataset.classes)):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], np.array(y_probs)[:, i])
    roc_auc[i] = roc_auc_score(y_true_bin[:, i], np.array(y_probs)[:, i])

# Plotting ROC for each class
for i in range(len(dataset.classes)):
    plt.plot(fpr[i], tpr[i], label=f"{dataset.classes[i]} (AUC = {roc_auc[i]:.2f})")

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Flower Classification')
plt.legend(loc='lower right')
plt.show()